In [1]:
"""
Created on August 4th 2022

@author: JJ
"""

'\nCreated on August 4th 2022\n\n@author: JJ\n'

In [2]:
import os, re, sys
import numpy as np
import pandas as pd
from math import log
import random
import scipy.io
import tensorflow as tf
import csv

2022-08-11 22:51:21.260632: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [3]:
# in a well-defined initial state.
random.seed
np.random.seed(1337)
tf.random.set_seed(1337)

In [4]:
import matplotlib
import matplotlib.pyplot as plt

In [51]:
from tensorflow.keras.optimizers import (
    RMSprop, 
    SGD, 
    Adam, 
    Nadam
)
from tensorflow.keras.models import (
    Sequential, 
    Model,
    load_model
)
from tensorflow.keras.layers import (
    Dense, 
    Dropout, 
    Activation, 
    Flatten,
    Conv1D,
    Conv2D,
    MaxPooling1D,
    MaxPooling2D,
    Bidirectional,
    Layer
)
from tensorflow.keras.layers import (
    Embedding,
    LSTM,
    Input,
    concatenate,
    multiply,
    Reshape,
    GRU,
)
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras.callbacks import (
    ModelCheckpoint, 
    EarlyStopping
)
from sklearn.metrics import (
    roc_curve,
    auc,
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
)
from tensorflow.keras.regularizers import (
    l2, 
    l1, 
    l1_l2
)
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import (
    activations, 
    initializers, 
    regularizers, 
    constraints
)

In [6]:
import os
num_threads = 4
# Maximum number of threads to use for OpenMP parallel regions.
os.environ["OMP_NUM_THREADS"] = "4"
# Without setting below 2 environment variables, it didn't work for me. Thanks to @cjw85 
os.environ["TF_NUM_INTRAOP_THREADS"] = "4"
os.environ["TF_NUM_INTEROP_THREADS"] = "4"

tf.config.threading.set_inter_op_parallelism_threads(
    num_threads
)
tf.config.threading.set_intra_op_parallelism_threads(
    num_threads
)
tf.config.set_soft_device_placement(True)

In [7]:
class Attention(Layer):
    def __init__(
        self,
        hidden,
        init="glorot_uniform",
        activation="linear",
        W_regularizer=None,
        b_regularizer=None,
        W_constraint=None,
        **kwargs
    ):
        self.init = initializers.get(init)
        self.activation = activations.get(activation)
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        self.W_constraint = constraints.get(W_constraint)
        self.hidden = hidden
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.input_length = input_shape[1]
        self.W0 = self.add_weight(
            name="{}_W1".format(self.name),
            shape=(input_dim, self.hidden),
            initializer="glorot_uniform",
            trainable=True,
        )  # Keras 2 API
        self.W = self.add_weight(
            name="{}_W".format(self.name),
            shape=(self.hidden, 1),
            initializer="glorot_uniform",
            trainable=True,
        )
        self.b0 = K.zeros((self.hidden,), name="{}_b0".format(self.name))
        self.b = K.zeros((1,), name="{}_b".format(self.name))
        # AttributeError: Can't set the attribute "trainable_weights",
        # likely because it conflicts with an existing read-only @property of the object.
        # Please choose a different name.
        # https://issueexplorer.com/issue/wenguanwang/ASNet/8
        self._trainable_weights = [self.W0, self.W, self.b, self.b0]

        self.regularizers = []
        if self.W_regularizer:
            self.W_regularizer.set_param(self.W)
            self.regularizers.append(self.W_regularizer)

        if self.b_regularizer:
            self.b_regularizer.set_param(self.b)
            self.regularizers.append(self.b_regularizer)

        self.constraints = {}
        if self.W_constraint:
            self.constraints[self.W0] = self.W_constraint
            self.constraints[self.W] = self.W_constraint

        super(Attention, self).build(input_shape)

    def call(self, x, mask=None):
        attmap = self.activation(K.dot(x, self.W0) + self.b0)
        attmap = K.dot(attmap, self.W) + self.b
        attmap = K.reshape(
            attmap, (-1, self.input_length)
        )  # Softmax needs one dimension
        attmap = K.softmax(attmap)
        dense_representation = K.batch_dot(attmap, x, axes=(1, 1))
        out = K.concatenate(
            [dense_representation, attmap]
        )  # Output the attention maps but do not pass it to the next layer by DIY flatten layer
        return out

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1] + input_shape[1])

    def get_config(self):
        config = {
            "init": "glorot_uniform",
            "activation": self.activation.__name__,
            "W_constraint": self.W_constraint.get_config()
            if self.W_constraint
            else None,
            "W_regularizer": self.W_regularizer.get_config()
            if self.W_regularizer
            else None,
            "b_regularizer": self.b_regularizer.get_config()
            if self.b_regularizer
            else None,
            "hidden": self.hidden if self.hidden else None,
        }
        base_config = super(Attention, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class attention_flatten(Layer):  # Based on the source code of Keras flatten
    def __init__(self, keep_dim, **kwargs):
        self.keep_dim = keep_dim # 64
        super(attention_flatten, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        if not all(input_shape[1:]):
            raise Exception(
                'The shape of the input to "Flatten" '
                "is not fully defined "
                "(got " + str(input_shape[2:]) + ". "
                'Make sure to pass a complete "input_shape" '
                'or "batch_input_shape" argument to the first '
                "layer in your model."
            )
        return (input_shape[0], self.keep_dim)  # Remove the attention map

    def call(self, x, mask=None):
        x = x[:, : self.keep_dim]
        #return K.batch_flatten(x)
        return x
    
    def get_config(self):
        config = super().get_config().copy()
        return config

In [8]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.utils import resample, shuffle
from sklearn.feature_selection import (
    SelectKBest,
    chi2
)
from collections import defaultdict

In [9]:
import pickle
with open ('fulldata_pseudoseq_dictionary.pkl', 'rb') as f:
    pseudo_seq = pickle.load(f)

In [10]:
type(pseudo_seq)

dict

In [11]:
import keras_tuner as kt

In [13]:
    training_data = []
    for allele in pseudo_seq.keys():
        allele_data = pseudo_seq[allele]
        random.shuffle(allele_data)
        allele_data = np.array(allele_data)
        training_data.extend(allele_data)
    
    [all_pep, all_mhc, all_target] = [[i[j] for i in training_data] for j in range(3)]
    all_pep = np.array(all_pep)
    all_mhc = np.array(all_mhc)
    all_target = np.array(all_target)

/tmp/ipykernel_227145/3434162491.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  allele_data = np.array(allele_data)


In [71]:
print(all_pep.shape)
print(all_mhc.shape)
print(all_target.shape)

(945434, 30, 20)
(945434,)
(945434,)


In [72]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1337)    
allprobas_=np.array([]) 
all_labels=np.array([])

In [66]:
def build_model(filter = 256, kernel = 9, kernel_constraint = 3,
				activation = 'relu', pool = 3, dropout1 = 0.6, 
				dropout2 = 0.5, lr = 0.001):

				input1 = Input(shape = (30,20), name = 'peptide')

				conv1  = Conv1D(filters = filter,
					kernel_size = kernel,
					padding = 'same', 
					activation = activation, 
					kernel_constraint = MaxNorm(kernel_constraint), name = 'conv1')

				conv1_pool_out = MaxPooling1D(pool_size = pool)

				conv1_pool_dropout_out = Dropout(dropout1)
				conv1_pool_dropout2_out = Dropout(dropout2)

				decoder  = Attention(hidden = 256, activation = 'linear')
				dense1   = Dense(1)

				output_1 = conv1(input1)
				output_1b = conv1_pool_out(output_1)
				output_2 = conv1_pool_dropout_out(output_1b)

				att_decoder = decoder(output_2)
				output_3 = attention_flatten(output_2.shape[2])(att_decoder)

				output_4 =  dense1(conv1_pool_dropout2_out(Flatten()(output_2)))

				output_all = concatenate([output_3, output_4])

				output_5 =  Dense(1)(output_all)
				output_f =  Activation('sigmoid')(output_5)


				model = Model(inputs = input1, outputs = output_f)
				opt = Adam(learning_rate= lr)
				model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy', "AUC"])
				model.summary()
				return model

In [ ]:
model = build_model()

In [39]:
es = EarlyStopping(monitor = 'val_accuracy', mode = 'max', verbose = 1, patience = 5)

In [73]:
for i, (train, test) in enumerate(kfold.split(all_pep, all_target)):
    training_pep = all_pep[train]
    training_mhc = all_mhc[train]
    training_target = all_target[train]
    
    validation_pep = all_pep[test]
    validation_mhc = all_mhc[test]
    validation_target = all_target[test]

In [ ]:
saved_model = model.fit([training_pep,training_mhc], 
            training_target,
            batch_size=512,
            epochs = 100,
            shuffle=True,
            callbacks=[es],
            validation_data=([validation_pep,validation_mhc], validation_target),
            verbose=1)

probas_ = saved_model.predict([np.array(validation_pep),np.array(validation_mhc)])
allprobas_ = np.append(allprobas_, probas_)           
all_labels = np.append(all_labels, validation_target)
del model